In [29]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

In [4]:
# Алгоритм Кнута-Морриса-Пратта
import time
def KnuthMorrisPratt(text,pattern):
    start_position = 0
    match = 0
    pattern = list(pattern) # таблица сдвигов
    shifts = [1] * (len(pattern) + 1)
    shift = 1
    for i in range(len(pattern)):
        while ((shift <= i) and (pattern[i] != pattern[i-shift])):
            shift += shifts[i-shift] # сдвиг по максимальному префиксу 
        shifts[i+1] = shift  
    for j in text: # ищем
        while match == len(pattern) or match >= 0 and pattern[match] != j: #  дошли до длины образа или не нашли совпадение
            start_position += shifts[match] # сдвигаем начальную позицию согласно таблице сдвигов
            match -= shifts[match]
        match += 1
        if match == len(pattern):
            print('Образ найден по индексу {}!'.format(start_position))
    if match != len(pattern):
        print('Nope!')

text = 'acdababaabbaaaabcd'
pattern = 'cd'
t = time.perf_counter()
KnuthMorrisPratt(text, pattern)
print('Алгоритм Кнута-Морриса-Пратта: {0:.6f} сек'.format((time.perf_counter()-t)))

Образ найден по индексу 1!
Образ найден по индексу 16!
Алгоритм Кнута-Морриса-Пратта: 0.007516 сек


In [24]:
# Алгоритм Бойера-Мура
def BoyerMoore(text, pattern):
    lst = list(pattern)
    sizePattern = len(lst)
    dct = {}
    for i in reversed(range(sizePattern-1)):
        if dct.get(lst[i]) == None:
            dct[lst[i]] = sizePattern - i - 1 
        dct['*'] = sizePattern # любой символ, который будет найдет в тексте, смещение будет равное длине образа
        
        size = len(text)
        if size >= sizePattern:
            head = sizePattern - 1 # счетчик проверяемого символа в строке
            while(head < size): # пока курсор не сдвинется за пределы текста
                k = 0
                j = 0
                ifBreak = False
                for j in range(sizePattern -1, -1,-1):
                    if text[head -k] != lst[j]:
                        if j== sizePattern - 1: # если несовпадающий символ последний в образе
                            off = dct[text[head]] if dct.get(text[head], False) else dct['*'] 
                        else:
                            off = dct[lst[j]] # если несовпадающий символ не последний в образе
                        head += off # смещение счётчика строки
                        ifBreak = True 
                        break
                    k += 1 
                if not ifBreak: # если дошли до начала образа - символы совпали
                    print(f'Образ найден по индексу {head - k +1}!')
                    break

        else:
            print('Pattern not found')
text = 'Do androids dream of electric sheep?'
pattern = 'droids'
print ("Учитывать регистр? Введите Y или N")
flag = input()
if (flag == "N"):
    text=text.lower()
    pattern=pattern.lower()

t = time.perf_counter()
BoyerMoore(text, pattern)
print('Алгоритм Бойера-Мура {0:.6f} сек'.format((time.perf_counter()-t)))

Учитывать регистр? Введите Y или N
N
Образ найден по индексу 5!
Образ найден по индексу 5!
Алгоритм Бойера-Мура 0.000223 сек


In [10]:
text = 'Do androids dream of electric sheep?'
pattern = 'droids'
t = time.perf_counter()
text.find(pattern)
print('Встроенный алгоритм поиска подстроки: {0:.6f} сек'.format((time.perf_counter()-t)))

Встроенный алгоритм поиска подстроки: 0.000077 сек


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [1]:
# ПЯТНАШКИ 
from queue import PriorityQueue
class Position:
    def __init__(self, position, start_distance,finish_distance):
        self.position = position
        self.start_distance = start_distance
        self.finish_distance = finish_distance

    def __str__(self):
        return '\n'.join((N*'{:3}').format(*[i%(N*N) for i in self.position[i:]]) for i in range(0, N*N, N))

    def __lt__(self, other):
        return self.start_distance+self.finish_distance < other.start_distance+other.finish_distance
N = 4 
def shifts(position): # сдвиги     
    zeroPosition = position.index(0) # нахождение нуля в массиве
    i, j = divmod(zeroPosition, N)
    displacement = [] # Варианты сдвига нуля
    if i > 0: displacement.append(-N) # up
    if i < N - 1: displacement.append(N) # down
    if j > 0: displacement.append(-1) # left
    if j < N - 1: displacement.append(1) # right
    for offset in displacement:
        swap = zeroPosition + offset # индексы новых позиций нуля
        yield tuple(position[swap] # новое поле после сдвига нуля
        if x==zeroPosition
        else position[zeroPosition]
        if x==swap
        else position[x]
        for x in range(N*N))

def parityOfPairs(state): # четность неправильных пар
    countOfPairs = 0
    for i in range(len(state)-1):
        if state[i] > state[i+1]:
            countOfPairs +=1
    return countOfPairs % 2
def Fifteen(startState):
    terminalState = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0) # если количество неправильных пар четное, то такая позиция нерешаема
    if parityOfPairs(startState)==0:
        print("Нет решений")
    else:
        startState= tuple(startState)
        tup = Position(startState, 0,0)
        print('Начальная позиция:')
        print(tup)
        fieldStates= PriorityQueue()
        fieldStates.put(tup)
        closePoints = set([tup]) # множество посещенных/закрытых вершин
        parents = {tup.position: None} # словарь в котором хранятся позиции и ее предыдущая позиция 
        while tup.position != terminalState:
            tup = fieldStates.get()
            for k in shifts(tup.position):
                count= 0
                if k not in closePoints: # если вершина еще не закрыта
                    for m in range(len(k)):
                        if k[m] != terminalState[m]:
                            count+=1
                    fieldStates.put(Position(k, tup.start_distance +1,tup.finish_distance+count))
                    parents[k] = tup # добавляем в словарь значение предыдущей этой позиции
                    closePoints.add(k) # закрываем вершину 
        steps = []
        previous = tup
        while tup.position != startState:
            tup = parents[tup.position]
            number = tup.position[previous.position.index(0)] # индекс нуля из текущего состояния и по этому индексу находится элемент, который сдвинули в предыдущем состоянии
            steps.append(number)
            previous = tup
        steps.reverse() # инвертирование пути, чтобы путь лежал от начала до конца
        print('Шаги:', steps)
startState = [1, 2, 3, 4, 5, 6, 7, 8, 9,10,12,15,13,14,11,0] # https://15puzzle.zone/ru/
Fifteen(startState)

Начальная позиция:
  1  2  3  4
  5  6  7  8
  9 10 12 15
 13 14 11  0
Шаги: [15, 12, 11, 15]
